In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [1]:
import pickle
import pandas as pd

In [2]:
with open('model.bin', 'rb') as f_in:
    (dv, model) = pickle.load(f_in)

In [3]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [4]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

In [5]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

# Question 1

In [6]:
y_pred.std()

5.28140357655334

# Question 2

In [8]:
year = 2022
month = 2

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [9]:
df_result = pd.DataFrame({'ride_id': df['ride_id'], 'prediction': y_pred})

In [12]:
output_file = "results.parquet"

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [1]:
!ls -ls

total 58632
    4 -rw-rw-r-- 1 pastor pastor      207 Jun 25 11:22 Pipfile
   12 -rw-r--r-- 1 pastor pastor     8966 Jun 25 11:22 Pipfile.lock
    4 -rw-rw-r-- 1 pastor pastor     3626 Jun 25 12:11 homework.ipynb
   20 -rw-rw-r-- 1 pastor pastor    17369 Jun 25 11:46 model.bin
58592 -rw-rw-r-- 1 pastor pastor 59994863 Jun 25 12:10 results.parquet
